In [31]:
import sys
import pandas as pd
import numpy as np
from enum import Enum, auto

In [32]:
# The following data is exported from the experiment sheet in .ods format
experiment_sheets = ["Experiment1","Experiment2","Experiment3","Experiment4","Experiment5","Experiment6"]
sim_data = pd.read_excel("exp_data.ods",header=[0,1], index_col=0, sheet_name=experiment_sheets)
real_data = pd.read_excel("exp_data.ods",header=[0,1], index_col=0, sheet_name=["Real"])
all_data = {**sim_data,**real_data}

# Comparison matrices definitions
#### We are going to create a scalar comparison matrix using a single metric `Traveled Distance`.
#### Two values are deemed the same if they are within $\delta$, where $\delta=0.05*maxVal$

In [33]:
# Data structure to keep output dataframes of comparison matrices
# metric = "driven_lanedir_consec"
metric = "in-drivable-lane"
# metric = "survival_time"
cm_dict = {}
for exp_name in all_data:
    # Proxy data variable
    input_df = all_data[exp_name]["Average"][metric]
    # Setup similarity treshold
    delta = 0.05*input_df.max()
    # Placeholder comparison matrix (df)
    cm = pd.DataFrame(np.zeros(([input_df.shape[0]]*2), dtype=float), index=input_df.index, columns=input_df.index)
    # Fill the comparison matrix
    for a1 in cm.index:
        for a2 in cm.columns:
            diff = input_df[a1] - input_df[a2]
            if (abs(diff) <= delta):
                #incomparable
                val = 0.5
            elif (diff > 0):
                #df[a1] is greater than df[a2]
                val = 1
            else:
                #df[a1] is lower than df[a2]
                assert(diff < 0)
                val = 0
            cm[a2][a1] = val
    cm_dict.update({exp_name:cm})

### For each simulator, compute the simple RPRG

In [34]:
rprg_dict = {}
for experiment in sim_data.keys():
    # Compute 1-norm between this exp's CM and the real CM
    sim_df = cm_dict[experiment]
    real_df = cm_dict["Real"]
    rprg = 0
    for a1 in sim_df.index:
        for a2 in sim_df.index:
            rprg += abs(sim_df[a1][a2] - real_df[a1][a2])
    rprg_dict.update({experiment:rprg})

In [35]:
# RPRG value for each simulator
rprg_dict

{'Experiment1': 121.0,
 'Experiment2': 96.0,
 'Experiment3': 134.0,
 'Experiment4': 107.0,
 'Experiment5': 130.0,
 'Experiment6': 134.0}

# Partial order for simulator

### Using Andrea's notes, using 3 metrics

In [36]:
# Define relation types
class Relations(Enum):
    BETTER = 1
    WORST = 2
    INDIFFERENT = 3
    INCOMPARABLE = 4
    
    def isSpecific(self):
        return self.value < 3
    
    def getScore(self):
        if (self.value == self.WORST.value):
            return 0
        elif (self.value == self.BETTER.value):
            return 1
        return 0.5
    
    @classmethod
    def areOpposed(cls,v1,v2):
        if (v1 is cls.BETTER and v2 is cls.WORST):
            return True
        elif (v2 is cls.BETTER and v1 is cls.WORST):
            return True
        else:
            return False
    
    @classmethod
    def compare(cls, sim, real):
        if (Relations.areOpposed(sim, real)):
            return Errors.TYPE1
        elif (not sim.isSpecific() and real.isSpecific()):
            return Errors.SPECIFICITY
        elif (sim.isSpecific() and not real.isSpecific()):
            return Errors.OVERCONFIDENCE
        else:
            return Errors.NOERROR
        
    
class Errors(Enum):
    NOERROR = 0
    OVERCONFIDENCE = 1
    SPECIFICITY = 2
    TYPE1 = 3
    
    def __lt__(self,other):
        return (self.value<other.value)

    def __le__(self,other):
        return(self.value<=other.value)

    def __gt__(self,other):
        return(self.value>other.value)

    def __ge__(self,other):
        return(self.value>=other.value)

    def __eq__(self,other):
        return (self is other)

    def __ne__(self,other):
        return not(self.__eq__(other))
    
    def __hash__(self):
        return hash(self.name)
    
#Tests
assert Relations.compare(sim=Relations.INCOMPARABLE, real=Relations.WORST) is Errors.SPECIFICITY
assert Relations.compare(sim=Relations.INCOMPARABLE, real=Relations.INDIFFERENT) is Errors.NOERROR
assert Relations.compare(sim=Relations.BETTER, real=Relations.WORST) is Errors.TYPE1
assert Relations.compare(sim=Relations.BETTER, real=Relations.INDIFFERENT) is Errors.OVERCONFIDENCE
assert Relations.compare(sim=Relations.BETTER, real=Relations.INCOMPARABLE) is Errors.OVERCONFIDENCE


In [37]:
# Build preference relations
pr_dict = {}
metrics = ["in-drivable-lane", "survival_time","driven_lanedir_consec"]
for exp_name in all_data:
    # Proxy data variable
    input_df = all_data[exp_name]["Average"]
    # Setup similarity treshold
#     delta = 0.05*input_df.max()
    # Placeholder preference relation matrix (df)
    pr = pd.DataFrame(np.zeros(([input_df.shape[0]]*2)), index=input_df.index, columns=input_df.index)
    # Fill the comparison matrix
    for a1 in pr.index:
        for a2 in pr.columns:
            metric_relation = []
            # For all metrics check the relation between a1 and a2
            for metric in metrics:
                delta = 0.05*input_df[metric].max()
                diff = input_df[metric][a1] - input_df[metric][a2]
                
                if (abs(diff) <= delta):
                    #incomparable
                    relation = Relations.INDIFFERENT
                elif (diff > 0):
                    #df[a1] is greater than df[a2]
                    relation = Relations.BETTER
                else:
                    #df[a1] is lower than df[a2]
                    assert(diff < 0)
                    relation = Relations.WORST
                metric_relation.append(relation)
            # if we are indifferent for all metric
            if (all([x is Relations.INDIFFERENT for x in metric_relation])):
                agents_relation = Relations.INDIFFERENT
                
            # if we are better or indifferent for all metric
            elif (all([x in [Relations.BETTER, Relations.INDIFFERENT] for x in metric_relation])):
                agents_relation = Relations.BETTER
            
            # if we are worst or indifferent
            elif (all([x in [Relations.WORST, Relations.INDIFFERENT] for x in metric_relation])):
                agents_relation = Relations.WORST
                  
            # if we have some best and some worst
            elif (Relations.WORST in metric_relation and Relations.BETTER in metric_relation):
                agents_relation = Relations.INCOMPARABLE
#             pr[a2][a1] = agents_relation
            pr.loc[a2,a1] = agents_relation
            
    pr_dict.update({exp_name:pr})

In [38]:
pr_dict["Experiment1"]

,fhebert,ayman,mokleit,yishu,daniil,himanshu,raphael,etienne,fernanda,js,phred,melisande,dishank,olga,bhavya,jerome
fhebert,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE
ayman,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE
mokleit,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE
yishu,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.BETTER,Relations.BETTER,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE
daniil,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER
himanshu,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE
raphael,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.WORST
etienne,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER
fernanda,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INDIFFERENT,Relations.WORST,Relations.WORST,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST
js,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER


In [39]:
# Now we compare the preference relations of a simulator to the real data

# for each pair of agent, the relation guessed by the simulator is either
em_dict = {}
for experiment in sim_data.keys():
    #  Proxy for experiment preference relation matrix
    pr_proxy = pr_dict[experiment]
    # Create the placeholder error matrix for this sim
    em = pd.DataFrame(np.zeros(([input_df.shape[0]]*2)), index=input_df.index, columns=input_df.index)
    
    for a1 in pr_proxy.index:
        for a2 in pr_proxy.columns:
            sim_pr = pr_proxy.at[a1,a2]
            real_pr = pr_dict["Real"].at[a1,a2]
            relation = Relations.compare(sim=sim_pr, real=real_pr)
            em.at[a1,a2] = relation
#             em.at[a1,a2] = Relations.compare(sim=pr_proxy.at[a1,a2], real=pr_dict["Real"].at[a1,a2])
    
    em_dict.update({experiment: em})
    

In [40]:
em_dict["Experiment2"]

,fhebert,ayman,mokleit,yishu,daniil,himanshu,raphael,etienne,fernanda,js,phred,melisande,dishank,olga,bhavya,jerome
fhebert,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.SPECIFICITY,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.SPECIFICITY
ayman,Errors.SPECIFICITY,Errors.NOERROR,Errors.TYPE1,Errors.SPECIFICITY,Errors.TYPE1,Errors.TYPE1,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.TYPE1,Errors.NOERROR,Errors.SPECIFICITY
mokleit,Errors.NOERROR,Errors.TYPE1,Errors.NOERROR,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.TYPE1,Errors.SPECIFICITY,Errors.NOERROR
yishu,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.SPECIFICITY,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR,Errors.SPECIFICITY,Errors.TYPE1
daniil,Errors.NOERROR,Errors.TYPE1,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.OVERCONFIDENCE,Errors.OVERCONFIDENCE,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR,Errors.TYPE1,Errors.NOERROR
himanshu,Errors.OVERCONFIDENCE,Errors.TYPE1,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.TYPE1,Errors.OVERCONFIDENCE
raphael,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.OVERCONFIDENCE,Errors.OVERCONFIDENCE,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.SPECIFICITY,Errors.OVERCONFIDENCE
etienne,Errors.OVERCONFIDENCE,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.SPECIFICITY,Errors.NOERROR
fernanda,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.OVERCONFIDENCE,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR
js,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.SPECIFICITY,Errors.SPECIFICITY,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.OVERCONFIDENCE,Errors.NOERROR,Errors.NOERROR,Errors.NOERROR,Errors.TYPE1,Errors.NOERROR,Errors.SPECIFICITY,Errors.OVERCONFIDENCE


## Evaluation of the simulators
##### The scoring should be adapted for each use. We will use

In [41]:
sim_performance = {}
for sim in em_dict.keys():
    unique, counts = np.unique(em_dict[sim], return_counts=True)
    sim_performance.update({sim:dict(zip(unique, counts))})
    
    

In [42]:
sim_performance

{'Experiment1': {<Errors.NOERROR: 0>: 110,
  <Errors.OVERCONFIDENCE: 1>: 46,
  <Errors.SPECIFICITY: 2>: 60,
  <Errors.TYPE1: 3>: 40},
 'Experiment2': {<Errors.NOERROR: 0>: 126,
  <Errors.OVERCONFIDENCE: 1>: 42,
  <Errors.SPECIFICITY: 2>: 70,
  <Errors.TYPE1: 3>: 18},
 'Experiment3': {<Errors.NOERROR: 0>: 104,
  <Errors.OVERCONFIDENCE: 1>: 50,
  <Errors.SPECIFICITY: 2>: 94,
  <Errors.TYPE1: 3>: 8},
 'Experiment4': {<Errors.NOERROR: 0>: 122,
  <Errors.OVERCONFIDENCE: 1>: 48,
  <Errors.SPECIFICITY: 2>: 48,
  <Errors.TYPE1: 3>: 38},
 'Experiment5': {<Errors.NOERROR: 0>: 124,
  <Errors.OVERCONFIDENCE: 1>: 26,
  <Errors.SPECIFICITY: 2>: 90,
  <Errors.TYPE1: 3>: 16},
 'Experiment6': {<Errors.NOERROR: 0>: 138,
  <Errors.OVERCONFIDENCE: 1>: 38,
  <Errors.SPECIFICITY: 2>: 60,
  <Errors.TYPE1: 3>: 20}}

## RPRG using 3 metrics

In [53]:
scorer = lambda x: x.getScore()
vfunc = np.vectorize(scorer)
rprg3_dict = {}

for experiment in sim_data.keys():
    # Compute 1-norm between this exp's CM and the real CM
    score_matrix = vfunc(pr_dict[experiment])
    rprg3 = np.sum(score_matrix)
    rprg3_dict.update({experiment:rprg3})

In [80]:
scorer = lambda x: x.getScore()
vfunc = np.vectorize(scorer)

# Now we compare the preference relations of a simulator to the real data
rprg3_dict = {}
for experiment in sim_data.keys():
    #  Proxy for experiment preference relation matrix
    df = pr_dict[experiment]
    pr_proxy = pd.DataFrame(vfunc(df), index=df.index, columns=df.index)
    real_proxy = pd.DataFrame(vfunc(pr_dict["Real"]), index=df.index, columns=df.index)
    # Create the placeholder error matrix for this sim
    rprg_mat = pd.DataFrame(np.zeros(([df.shape[0]]*2)), index=df.index, columns=df.index)
    rprg3 = 0
    for a1 in pr_proxy.index:
        for a2 in pr_proxy.columns:
            sim_pr = pr_proxy.at[a1,a2]
            real_pr = real_proxy.at[a1,a2]
            rprg3 += abs(sim_pr - real_pr)
    rprg3_dict.update({experiment: rprg3})


In [81]:
rprg3_dict

{'Experiment1': 93.0,
 'Experiment2': 74.0,
 'Experiment3': 80.0,
 'Experiment4': 86.0,
 'Experiment5': 74.0,
 'Experiment6': 69.0}

# Workspace

In [75]:
pr_dict

{'Experiment1':                           fhebert                   ayman  \
 fhebert     Relations.INDIFFERENT  Relations.INCOMPARABLE   
 ayman      Relations.INCOMPARABLE   Relations.INDIFFERENT   
 mokleit    Relations.INCOMPARABLE        Relations.BETTER   
 yishu            Relations.BETTER        Relations.BETTER   
 daniil           Relations.BETTER        Relations.BETTER   
 himanshu         Relations.BETTER  Relations.INCOMPARABLE   
 raphael    Relations.INCOMPARABLE  Relations.INCOMPARABLE   
 etienne          Relations.BETTER        Relations.BETTER   
 fernanda   Relations.INCOMPARABLE  Relations.INCOMPARABLE   
 js               Relations.BETTER  Relations.INCOMPARABLE   
 phred      Relations.INCOMPARABLE  Relations.INCOMPARABLE   
 melisande  Relations.INCOMPARABLE  Relations.INCOMPARABLE   
 dishank    Relations.INCOMPARABLE  Relations.INCOMPARABLE   
 olga             Relations.BETTER        Relations.BETTER   
 bhavya            Relations.WORST  Relations.INCOMPARA

In [65]:
vfunc(pr_dict["Experiment1"])[0]

array([0.5, 0.5, 0.5, 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0.5, 0.5, 0.5,
       0. , 1. , 0.5])

In [64]:
np.sum(vfunc(pr_dict["Experiment2"])[0])

array([0.5, 0.5, 0.5, 0. , 0. , 0. , 0.5, 0. , 0.5, 0.5, 0.5, 0.5, 0.5,
       0. , 0. , 0.5])

In [70]:
b = 0
for i in range(16):
    a = np.sum(vfunc(pr_dict["Experiment1"])[i])
    b+=a
    print(f"a: {a}, b: {b}")

a: 5.5, b: 5.5
a: 5.5, b: 11.0
a: 8.0, b: 19.0
a: 10.0, b: 29.0
a: 13.0, b: 42.0
a: 7.0, b: 49.0
a: 5.5, b: 54.5
a: 12.5, b: 67.0
a: 3.5, b: 70.5
a: 11.5, b: 82.0
a: 8.5, b: 90.5
a: 7.0, b: 97.5
a: 8.5, b: 106.0
a: 11.5, b: 117.5
a: 2.5, b: 120.0
a: 8.0, b: 128.0


In [71]:
b = 0
for i in range(16):
    a = np.sum(vfunc(pr_dict["Experiment2"])[i])
    b+=a
    print(f"a: {a}, b: {b}")

a: 5.0, b: 5.0
a: 6.0, b: 11.0
a: 7.0, b: 18.0
a: 8.5, b: 26.5
a: 12.0, b: 38.5
a: 12.5, b: 51.0
a: 6.5, b: 57.5
a: 8.0, b: 65.5
a: 3.5, b: 69.0
a: 10.0, b: 79.0
a: 10.5, b: 89.5
a: 9.5, b: 99.0
a: 7.5, b: 106.5
a: 9.5, b: 116.0
a: 8.5, b: 124.5
a: 3.5, b: 128.0


In [72]:
pr_dict["Experiment1"]

,fhebert,ayman,mokleit,yishu,daniil,himanshu,raphael,etienne,fernanda,js,phred,melisande,dishank,olga,bhavya,jerome
fhebert,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE
ayman,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE
mokleit,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE
yishu,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.BETTER,Relations.BETTER,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE
daniil,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER
himanshu,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE
raphael,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.WORST
etienne,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER
fernanda,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INDIFFERENT,Relations.WORST,Relations.WORST,Relations.WORST,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST
js,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER


In [46]:

tt = vfunc(test)

In [30]:
scorer(Relations.BETTER)

1

In [47]:
tt

array([[0.5, 0. , 0.5, 0. , 0. , 0.5, 0. , 0.5, 0.5, 0. , 0.5, 0.5, 0. ,
        0.5, 0. , 0. ],
       [1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. , 0.5, 1. ],
       [0.5, 0. , 0.5, 0.5, 0.5, 0. , 0.5, 1. , 1. , 0.5, 1. , 1. , 0.5,
        1. , 0. , 0.5],
       [1. , 0. , 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5, 0. , 0.5, 0.5, 0. ,
        0.5, 0. , 0. ],
       [1. , 0. , 0.5, 1. , 0.5, 0.5, 1. , 0.5, 0.5, 1. , 0.5, 0.5, 1. ,
        0.5, 0. , 1. ],
       [0.5, 0. , 1. , 0.5, 0.5, 0.5, 0.5, 1. , 1. , 0.5, 1. , 1. , 0.5,
        1. , 0. , 0.5],
       [1. , 0. , 0.5, 1. , 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0.5, 0. , 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5,
        0.5, 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0.5, 0. , 0.5, 0. , 0.5, 0.5, 0. , 0. , 0.5,
        0. , 0. , 0.5],
       [1. , 0. , 0.5, 1. , 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0. ,
        0.5, 0. , 0.5],
       [0.5, 0. , 0. , 0.5, 0.

In [48]:
test

,fhebert,ayman,mokleit,yishu,daniil,himanshu,raphael,etienne,fernanda,js,phred,melisande,dishank,olga,bhavya,jerome
fhebert,Relations.INDIFFERENT,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST
ayman,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.BETTER,Relations.INDIFFERENT,Relations.BETTER
mokleit,Relations.INCOMPARABLE,Relations.WORST,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE
yishu,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST
daniil,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.BETTER,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.BETTER,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER
himanshu,Relations.INCOMPARABLE,Relations.WORST,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.BETTER,Relations.INCOMPARABLE,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE
raphael,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE
etienne,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.BETTER,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE
fernanda,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.WORST,Relations.INCOMPARABLE
js,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.BETTER,Relations.WORST,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.INDIFFERENT,Relations.INCOMPARABLE,Relations.INCOMPARABLE,Relations.WORST,Relations.INCOMPARABLE,Relations.WORST,Relations.INDIFFERENT


In [50]:
a = np.array([[1,2],[3,4]])

In [51]:
a

array([[1, 2],
       [3, 4]])

In [52]:
np.sum(a)

10